## Movie Recommendation System

Based on the tutorial - https://www.youtube.com/watch?v=9gBC9R-msAk&index=3&list=PL2-dafEMk2A6QKz1mrk1uIGfHkC1zZ6UU

In [35]:
import numpy as np

In [36]:
import scipy as sp

In [37]:
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

In [38]:
data = fetch_movielens(min_rating=4.0)

In [39]:
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<type 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<type 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [40]:
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)



CPU times: user 1.12 s, sys: 14.1 ms, total: 1.14 s
Wall time: 1.15 s


In [41]:
from lightfm.evaluation import precision_at_k

print("Train precision : %.2f" % precision_at_k(model, data['train'], k=10).mean())
print("Test precision : %.2f" % precision_at_k(model, data['test'], k=10).mean())

Train precision : 0.52
Test precision : 0.08


In [42]:
def sample_recommendation(user_ids, model, data) :
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("   Known positives : ")
        
        for x in known_positives[:5] :
            print("    %s" % x)
            
        print("   Recommended : ")
        
        for x in top_items[:5] :
            print("    %s" % x)
            
sample_recommendation([3, 20, 14, 35], model, data)

User 3
   Known positives : 
    Seven (Se7en) (1995)
    Contact (1997)
    Starship Troopers (1997)
    Air Force One (1997)
    In & Out (1997)
   Recommended : 
    Scream (1996)
    G.I. Jane (1997)
    Starship Troopers (1997)
    Kiss the Girls (1997)
    Air Force One (1997)
User 20
   Known positives : 
    Toy Story (1995)
    Twelve Monkeys (1995)
    Dead Man Walking (1995)
    Mr. Holland's Opus (1995)
    From Dusk Till Dawn (1996)
   Recommended : 
    Scream (1996)
    Godfather, The (1972)
    Pulp Fiction (1994)
    Heat (1995)
    GoodFellas (1990)
User 14
   Known positives : 
    Dead Man Walking (1995)
    Postino, Il (1994)
    Mr. Holland's Opus (1995)
    Star Wars (1977)
    Truth About Cats & Dogs, The (1996)
   Recommended : 
    English Patient, The (1996)
    Sense and Sensibility (1995)
    Air Force One (1997)
    Titanic (1997)
    Contact (1997)
User 35
   Known positives : 
    Air Bud (1997)
    Scream (1996)
    Rainmaker, The (1997)
    Game, The (